                                  GESTURE CONTROLLED SMART HOME USING MEDIAPIPE


This project aims to revolutionize our interaction with the environment. 'Gesture-Controlled Smart Home using MediaPipe' offers a glimpse into a future where gestures become commands. Identified specific hand movements for controlling lights, fans, and more, making your home smarter and more interactive. While physical device integration wasn't feasible in this project, the concept can easily translate into practical applications using various smart home or IoT devices.

In [1]:
#Importing necessary libraries

import cv2
import mediapipe as mp
import time
import pyttsx3
import threading

In [10]:
# CONTROLLING VARIOUS DEVICES


# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize MediaPipe Hands
hands = mp_hands.Hands(max_num_hands=1)

# Open the webcam
video = cv2.VideoCapture(0)

# Variable to track the previous state of the hand (open or closed)
previous_hand_state = "closed"  # Initially, assume hand is closed
lights_state = "off"  # Initially, assume lights are off

# Initialize the text-to-speech engine
engine = pyttsx3.init()

def speak_command(command):
    engine.say(command)
    engine.runAndWait()

while True:
    # Read a frame from the webcam
    success, image = video.read()
    image = cv2.flip(image, 1)  # Flip horizontally for a laterally inverted view
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(image_rgb)

    tipids = [4, 8, 12, 16, 20]
    lmlist = []


    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for id, lm in enumerate(hand_landmarks.landmark):
                # Append landmark positions to lmlist
                lmlist.append([id, lm.x, lm.y, lm.z])

                if len(lmlist) != 0 and len(lmlist) == 21:
                    fingerlist = []
                    
                    # Thumb
                    if lmlist[tipids[0]][1] < lmlist[tipids[0] - 1][1]:
                        fingerlist.append(1)
                    else:
                        fingerlist.append(0)

                    # For other fingers
                    for i in range(1, 5):
                        if lmlist[tipids[i]][2] < lmlist[tipids[i] - 2][2]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)

                    # Determine the number of fingers extended
                    num_fingers_extended = sum(fingerlist)

                    # Determine if the thumb is open
                    thumb_open = lmlist[tipids[0]][1] < lmlist[tipids[0] - 1][1]

                    # Perform actions based on the number of fingers extended and hand state
                    if num_fingers_extended == 5 or (num_fingers_extended == 4 and thumb_open):
                        cv2.putText(image, "Lights On", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 60, 245), 2, cv2.LINE_AA)
                        threading.Thread(target=speak_command, args=("Lights On",)).start()
                        lights_state = "on"
                    elif num_fingers_extended == 0:
                        cv2.putText(image, "Lights Off", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 60, 245), 2, cv2.LINE_AA)
                        threading.Thread(target=speak_command, args=("Lights off",)).start()
                        lights_state = "off"
                    elif num_fingers_extended == 1:
                        cv2.putText(image, "Turning on TV", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 60, 245), 2, cv2.LINE_AA)
                        threading.Thread(target=speak_command, args=("Turning on TV",)).start()
                    elif num_fingers_extended == 2:
                        cv2.putText(image, "Turning on Fan", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 60, 245), 2, cv2.LINE_AA)
                        threading.Thread(target=speak_command, args=("Turning on Fan",)).start()
                    elif num_fingers_extended == 3:
                        cv2.putText(image, "Turning on AC", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 60, 245), 2, cv2.LINE_AA)
                        threading.Thread(target=speak_command, args=("Turning on AC",)).start()
                    elif num_fingers_extended == 4:
                        cv2.putText(image, "Turning on Oven", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 60, 245), 2, cv2.LINE_AA)
                        threading.Thread(target=speak_command, args=("Turning on Oven",)).start()
                        
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=3),
                                      mp_drawing.DrawingSpec(color=(0, 200, 0), thickness=3))

    # Display the frame
    cv2.imshow('Hand Gesture Control', image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close OpenCV windows
video.release()
cv2.destroyAllWindows()

Exception in thread Thread-417 (speak_command):
Traceback (most recent call last):
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_15192\3629623068.py", line 23, in speak_command
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-418 (speak_command):
Traceback (most recent call last):
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._a

In [11]:
#TV CONTROLLING MODE 

x1 = y1 = x2 = y2 = 0
current_volume = 20 # Initial volume level
current_channel = 1  # Initial channel

initial_x1=None
initial_y1=None

webcam = cv2.VideoCapture(0)
my_hands = mp.solutions.hands.Hands(max_num_hands=1)  # Detect only one hand
drawing_utils = mp.solutions.drawing_utils

channel_change_threshold = 100  # Threshold for channel change (adjust as needed)
initial_x1 = 0  # Store the initial x1 position
channel_change_start_time = 0  # Timestamp when channel change started
channel_change_duration = 3  # Duration (in seconds) to change one channel

thumb_and_index_extended = False  # Flag to track thumb and index finger extension

# Define a threshold for thumb and index finger distance
volume_control_threshold = 40

while True:
    _, image = webcam.read()
    image = cv2.flip(image, 1)
    frame_height, frame_width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    output = my_hands.process(rgb_image)
    hands = output.multi_hand_landmarks

    if hands:
        for hand in hands:
            drawing_utils.draw_landmarks(image, hand)
            landmarks = hand.landmark
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 8:
                    cv2.circle(img=image, center=(x, y), radius=8, color=(0, 255, 255), thickness=3)
                    if initial_x1 == 0:
                        initial_x1 = x
                    x1 = x
                    y1 = y
                if id == 4:
                    cv2.circle(img=image, center=(x, y), radius=8, color=(255, 0, 255), thickness=3)
                    x2 = x
                    y2 = y

        dist = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5 // 4

        # Detect thumb and index finger extension for volume control
        if landmarks[4].y < landmarks[3].y and landmarks[8].y < landmarks[7].y:
            thumb_and_index_extended = True
        else:
            thumb_and_index_extended = False

        # Adjust volume when thumb and index finger are extended
        if thumb_and_index_extended:
            if dist > 20 and dist < volume_control_threshold and current_volume < 100:
                current_volume += 1  # Increase volume slowly
            elif dist <= 20 and dist < volume_control_threshold and current_volume > 0:
                current_volume -= 1  # Decrease volume slowly

        # Detect swiping right to change channels based on initial_x1 position
        if x1 > initial_x1 + channel_change_threshold and time.time() - channel_change_start_time > channel_change_duration:
            cv2.putText(image, "Swipe Right to Change Channel", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                        cv2.LINE_AA)
            current_channel += 1  # Change channel when swiping right
            channel_change_start_time = time.time()  # Reset the channel change timer

        # Detect swiping left to change channels
        elif x1 < initial_x1 - channel_change_threshold and time.time() - channel_change_start_time > channel_change_duration:
            cv2.putText(image, "Swipe Left to Change Channel", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                        cv2.LINE_AA)
            current_channel -= 1  # Change channel when swiping left
            channel_change_start_time = time.time()  # Reset the channel change timer
        
    # Display the current volume and channel
    cv2.putText(image, f"TV Control Mode", (10, 50), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (80, 60, 245), 2,
                cv2.LINE_AA)
    cv2.putText(image, f"Volume: {current_volume}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                cv2.LINE_AA)
    cv2.putText(image, f"Channel: {current_channel}", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                cv2.LINE_AA)

    cv2.imshow('hand control', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

In [12]:
# AC CONTROLLING MODE
import cv2
import mediapipe as mp
import time

ac_temperature = 4  # Initial AC temperature setting
video = cv2.VideoCapture(0)
my_hands = mp.solutions.hands.Hands(max_num_hands=1)  # Detect only one hand
drawing_utils = mp.solutions.drawing_utils
change=0.1
temperature_change_interval = 2  # Delay interval between temperature changes (in seconds)
last_temperature_change_time = 0  # Initialize the last change time

while True:
    _, image = video.read()
    image = cv2.flip(image, 1)
    frame_height, frame_width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    output = my_hands.process(rgb_image)
    hands = output.multi_hand_landmarks

    current_time = time.time()  # Get the current time

    # Check if enough time has passed to allow another temperature change
    if current_time - last_temperature_change_time >= temperature_change_interval:


        if hands:
            for hand in hands:
                drawing_utils.draw_landmarks(image, hand)
                landmarks = hand.landmark
                for id, landmark in enumerate(landmarks):
                    x = int(landmark.x * frame_width)
                    y = int(landmark.y * frame_height)

                    # Detect thumbs up gesture to increase temperature
                    if id == 4 and landmarks[4].y < landmarks[3].y:
                        cv2.circle(img=image, center=(x, y), radius=8, color=(0, 255, 0), thickness=3)
                        ac_temperature += change  # Increase temperature

                    # Detect thumbs down gesture to decrease temperature
                    if id == 4 and landmarks[4].y > landmarks[3].y:
                        cv2.circle(img=image, center=(x, y), radius=8, color=(0, 0, 255), thickness=3)
                        ac_temperature -= change  # Decrease temperature

            # Update the time of the last temperature change
            last_temperature_change_time = current_time

    # Display the current AC temperature setting
    cv2.putText(image, f"AC Control Mode", (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (80, 60, 245), 2,
                cv2.LINE_AA)
    cv2.putText(image, f"Temperature: {ac_temperature:.1f}degreeC", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                cv2.LINE_AA)

    cv2.imshow('hand control', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

In [13]:
#LIGHT CONTROLLING MODE

# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize MediaPipe Hands
hands = mp_hands.Hands(max_num_hands=1)

light_brightness = 50  # Initial light brightness (0-100 scale)
light_color = "White"  # Initial light color (white)
change = 1 # Brightness change amount (adjust as needed)
color_change_interval = 2  # Delay interval between color changes (in seconds)

color_options = [
    "White",  
    "Red",    
    "Green",  
    "Yellow", 
    "Blue"    
]
current_color_option = 0  # Index of the current color option

last_color_change_time = 0  # Initialize the last color change time

# Open the webcam
video = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    success, image = video.read()
    image = cv2.flip(image, 1)  # Flip horizontally for a laterally inverted view
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(image_rgb)

    tipids = [4, 8, 12, 16, 20]
    lmlist = []
    current_time = time.time()  # Get the current time
    
    # Check if enough time has passed to allow another brightness change
    if current_time - last_color_change_time >= color_change_interval:

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for id, lm in enumerate(hand_landmarks.landmark):
                # Append landmark positions to lmlist
                    lmlist.append([id, lm.x, lm.y, lm.z])

                    if len(lmlist) != 0 and len(lmlist) == 21:
                        fingerlist = []
                    
                        # Thumb
                        if lmlist[tipids[0]][1] < lmlist[tipids[0] - 1][1]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)

                        # For other fingers
                        for i in range(1, 5):
                            if lmlist[tipids[i]][2] < lmlist[tipids[i] - 2][2]:
                                fingerlist.append(1)
                            else:
                                fingerlist.append(0)
                         # Determine the number of fingers extended
                        num_fingers_extended = sum(fingerlist)

                        if (
                             lmlist[tipids[1]][2] < lmlist[tipids[1] - 2][2]
                             and lmlist[tipids[4]][2] < lmlist[tipids[4] - 2][2]
                             and num_fingers_extended == 2
                             ):
                            current_color_option = (current_color_option + 1) % len(color_options)
                            light_color = color_options[current_color_option]

                        # Determine if the thumb is open
                        thumb_open = lmlist[tipids[0]][1] < lmlist[tipids[0] - 1][1]

                        # Perform actions based on the number of fingers extended and hand state
                        if num_fingers_extended == 5 or (num_fingers_extended == 4 and thumb_open):
                            light_brightness+=change
                        if num_fingers_extended == 0 :
                            light_brightness-=change
        last_color_change_time =current_time                    

    light_color_bgr = (0, 0, 0)  # Default color for text (black)
    if light_color == "White":
        light_color_bgr = (255, 255, 255)
    elif light_color == "Red":
        light_color_bgr = (0, 0, 255)
    elif light_color == "Green":
        light_color_bgr = (0, 255, 0)
    elif light_color == "Yellow":
        light_color_bgr = (0, 255, 255)
    elif light_color == "Blue":
        light_color_bgr = (255, 0, 0)

    # Display the current AC temperature setting
    cv2.putText(image, f"Light Control Mode", (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (80, 60, 245), 3,
                cv2.LINE_AA)
    cv2.putText(image, f"Brightness:", (20, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (234, 44, 24), 2,
                cv2.LINE_AA)        
    cv2.putText(image, f"Light Color:", (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (234, 44, 24), 2,
                cv2.LINE_AA)  
    cv2.putText(image, f"{light_color}", (210, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, light_color_bgr, 2,
                cv2.LINE_AA) 
    # Simulate controlling a light with brightness and color settings
    simulated_light = cv2.circle(
        image, (frame_width - 400,200), 40, light_color_bgr, -1
    )  # Simulate a colored light
    cv2.putText(
        simulated_light,
        f"{light_brightness}%",
        (frame_width - 415, 205),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 0, 0),
        1,
        cv2.LINE_AA,
    )  # Display brightness on the light
    # Display the frame
    cv2.imshow('Hand Gesture Control', image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close OpenCV windows
video.release()
cv2.destroyAllWindows()

In [14]:
import cv2
import mediapipe as mp
import time

x1 = y1 = x2 = y2 = 0
current_volume = 30 # Initial volume level
current_channel = 1  # Initial channel

initial_x1 = None
initial_y1 = None

channel_change_threshold = 40  # Threshold for channel change (adjust as needed
channel_change_start_time = 0  # Timestamp when channel change started
channel_change_duration = 5  # Duration (in seconds) to change one channel

thumb_and_index_extended = False  # Flag to track thumb and index finger extension
# Define a threshold for thumb and index finger distance
volume_control_threshold = 50

# Variable to track the previous state of the hand (open or closed)
previous_hand_state = "closed"  # Initially, assume hand is closed
lights_state = "off"  # Initially, assume lights are off

video = cv2.VideoCapture(0)
my_hands = mp.solutions.hands.Hands(max_num_hands=1)  # Detect only one hand
drawing_utils = mp.solutions.drawing_utils


while True:
    if initial_x1 is None:
        initial_x1 = 0
    # Read a frame from the webcam
    success, image = video.read()
    image = cv2.flip(image, 1)  # Flip horizontally for a laterally inverted view
    frame_height,frame_width,_=image.shape

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = my_hands.process(image_rgb)

    tipids = [4, 8, 12, 16, 20]
    lmlist = []


    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for id, lm in enumerate(hand_landmarks.landmark):
                # Append landmark positions to lmlist
                lmlist.append([id, lm.x, lm.y, lm.z])

                if len(lmlist) != 0 and len(lmlist) == 21:
                    fingerlist = []
                    
                    # Thumb
                    if lmlist[tipids[0]][1] < lmlist[tipids[0] - 1][1]:
                        fingerlist.append(1)
                    else:
                        fingerlist.append(0)

                    # For other fingers
                    for i in range(1, 5):
                        if lmlist[tipids[i]][2] < lmlist[tipids[i] - 2][2]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)

                    # Determine the number of fingers extended
                    num_fingers_extended = sum(fingerlist)

                    # Determine if the thumb is open
                    thumb_open = lmlist[tipids[0]][1] < lmlist[tipids[0] - 1][1]

                    # Perform actions based on the number of fingers extended and hand state
                    if num_fingers_extended == 5 or (num_fingers_extended == 4 and thumb_open):
                        cv2.putText(image, "Lights On", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        lights_state = "on"
                    elif num_fingers_extended == 0:
                        cv2.putText(image, "Lights Off", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        lights_state = "off"
                    elif num_fingers_extended == 1:
                        cv2.putText(image, "Turning on TV", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        if results.multi_hand_landmarks:
                            for hand_landmarks in results.multi_hand_landmarks:
                                drawing_utils.draw_landmarks(image, hand_landmarks)
                                landmarks = hand_landmarks.landmark
                                for id, landmark in enumerate(landmarks):
                                    x = int(landmark.x * frame_width)
                                    y = int(landmark.y * frame_height)
                                    if id == 8:
                                         cv2.circle(img=image, center=(x, y), radius=8, color=(0, 255, 255), thickness=3)
                                    if initial_x1 == 0:
                                         initial_x1 = x
                                         x1 = x
                                         y1 = y
                                    if id == 4:
                                         cv2.circle(img=image, center=(x, y), radius=8, color=(255, 0, 255), thickness=3)
                                         x2 = x
                                         y2 = y

                            dist = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5 // 4

                            # Detect thumb and index finger extension for volume control
                            if landmarks[4].y < landmarks[3].y and landmarks[8].y < landmarks[7].y:
                                thumb_and_index_extended = True
                            else:
                                thumb_and_index_extended = False
                            # Adjust volume when thumb and index finger are extended
                            if thumb_and_index_extended:
                                if dist > 20 and dist < volume_control_threshold and current_volume < 100:
                                     current_volume += 1  # Increase volume slowly
                                elif dist <= 20 and dist < volume_control_threshold and current_volume > 0:
                                     current_volume -= 1  # Decrease volume slowly

                           # Detect swiping right to change channels based on initial_x1 position
                            if x1 > initial_x1 + channel_change_threshold and time.time() - channel_change_start_time > channel_change_duration:
                                cv2.putText(image, "Swipe Right to Change Channel", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                                  cv2.LINE_AA)
                                current_channel += 1  # Change channel when swiping right
                                channel_change_start_time = time.time()  # Reset the channel change timer

                             # Detect swiping left to change channels
                            elif x1 < initial_x1 - channel_change_threshold and time.time() - channel_change_start_time > channel_change_duration:
                                cv2.putText(image, "Swipe Left to Change Channel", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                                cv2.LINE_AA)
                                current_channel -= 1  # Change channel when swiping left
                                channel_change_start_time = time.time()  # Reset the channel change timer
                            if num_fingers_extended == 0:
                                break

                        # Display the current volume and channel
                        cv2.putText(image, f"Volume: {current_volume}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                              cv2.LINE_AA)
                        cv2.putText(image, f"Channel: {current_channel}", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                           cv2.LINE_AA)
                                
                    elif num_fingers_extended == 2 and lmlist[8][1] < lmlist[7][1] and lmlist[12][1] < lmlist[11][1]:
                        cv2.putText(image, "Turning on Fan", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    elif num_fingers_extended == 3 and lmlist[8][1] < lmlist[7][1] and lmlist[12][1] < lmlist[11][1] and lmlist[16][1] < lmlist[15][1]:
                        cv2.putText(image, "Turning on AC", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    elif num_fingers_extended == 4 and thumb_open==False:
                        cv2.putText(image, "Turning on Oven", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        
            drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS,
                                      drawing_utils.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=3),
                                      drawing_utils.DrawingSpec(color=(0, 200, 0), thickness=3))

    # Display the frame
    cv2.imshow('Hand Gesture Control', image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close OpenCV windows
video.release()
cv2.destroyAllWindows()